為了運行以下筆記本，如果你還沒有設定，需要在 .env 文件中將 openai 金鑰設為 `OPENAI_API_KEY`。


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

接下來，我們將把嵌入索引載入到 Pandas 資料框中。嵌入索引儲存在一個名為 `embedding_index_3m.json` 的 JSON 檔案中。嵌入索引包含截至 2023 年 10 月底的每個 YouTube 逐字稿的嵌入向量。


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

接著，我們將建立一個名為 `get_videos` 的函數，用來在嵌入索引中搜尋查詢。該函數會回傳與查詢最相似的前 5 部影片。函數的運作方式如下：

1. 首先，建立嵌入索引的副本。
2. 接著，使用 OpenAI 嵌入 API 計算查詢的嵌入向量。
3. 然後，在嵌入索引中建立一個名為 `similarity` 的新欄位。`similarity` 欄位包含查詢嵌入向量與每個影片片段嵌入向量之間的餘弦相似度。
4. 接著，根據 `similarity` 欄位篩選嵌入索引。只保留餘弦相似度大於或等於 0.75 的影片。
5. 最後，根據 `similarity` 欄位排序嵌入索引，並回傳前 5 部影片。


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

這個函數非常簡單，它只是打印出搜索查詢的結果。


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. 首先，將嵌入索引加載到 Pandas Dataframe 中。
2. 接著，系統會提示用戶輸入查詢。
3. 然後呼叫 `get_videos` 函數在嵌入索引中搜尋查詢內容。
4. 最後呼叫 `display_results` 函數將結果顯示給用戶。
5. 接著系統會提示用戶輸入另一個查詢。此過程會持續，直到用戶輸入 `exit`。

![](../../../../translated_images/mo/notebook-search.1e320b9c7fcbb0bc.webp)

系統會提示你輸入查詢。輸入查詢後按下 Enter。應用程式會返回與查詢相關的影片清單。應用程式也會返回影片中答案所在位置的連結。

以下是一些可嘗試的查詢：

- 甚麼是 Azure Machine Learning？
- 卷積神經網絡如何運作？
- 甚麼是神經網絡？
- 我可以在 Azure Machine Learning 中使用 Jupyter Notebooks 嗎？
- 甚麼是 ONNX？


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：
本文件係使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我哋致力於確保準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件之母語版本應視為權威來源。對於重要資訊，建議採用專業人工翻譯。我哋對因使用本翻譯而引致之任何誤解或誤釋概不負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
